# Kernel methods for a 2 dimensional elliptic PDE

Here we will solve the PDE

$$\begin{cases} C_1 \Delta u + C_2 u^3 = f, & \boldsymbol{x} \in [0,1]^d \\ u = 0, &\boldsymbol{x} \in \partial [0,1]^d \end{cases}$$

where $\Delta$ is the Laplacian. 

## Imports

### 2 dimensional 

For the two dimensional problem we set 
$$u(\boldsymbol{x}) = \exp\left(-\frac{1}{1-r^2}\right)\left[\sin(\pi x_1)\sin(\pi x_2)+4\sin(4\pi x_1)\sin(4 \pi x_2)\right] 1_{\boldsymbol{x} \in (0,1)^2}$$
where $r = \sqrt{(2x_1-1)^2+(2x_2-1)^2}$

In [ ]:
def u_2d(x):
    x1,x2 = x[:,0],x[:,1]
    y = torch.zeros_like(x1)
    r = torch.sqrt((2*x1-1)**2+(2*x2-1)**2)
    b = r<1
    t1 = torch.exp(-1/(1-r[b]**2))
    t2 = torch.sin(torch.pi*x1[b])*torch.sin(torch.pi*x2[b])
    t3 = 4*torch.sin(6*torch.pi*x1[b])*torch.sin(6*torch.pi*x2[b])
    y[b] = t1*(t2+t3)
    return y
def u_laplace_2d(x):
    x1g,x2g = x[:,0].clone().requires_grad_(),x[:,1].clone().requires_grad_()
    xg = torch.hstack([x1g[:,None],x2g[:,None]])
    yg = u_2d(xg)
    grad_outputs = torch.ones_like(yg,requires_grad=False)
    yp1g = torch.autograd.grad(yg,x1g,grad_outputs,create_graph=True)[0]
    yp1p1g = torch.autograd.grad(yp1g,x1g,grad_outputs,create_graph=True)[0]
    yp2g = torch.autograd.grad(yg,x2g,grad_outputs,create_graph=True)[0]
    yp2p2g = torch.autograd.grad(yp2g,x2g,grad_outputs,create_graph=True)[0]
    return (yp1p1g+yp2p2g).detach()
def f_2d(x, C1=-1/250, C2=-1):
    return C1*u_laplace_2d(x)+C2*u_2d(x)**3
x1dticks = torch.linspace(0,1,65,dtype=float)
x1mesh,x2mesh = torch.meshgrid(x1dticks,x1dticks,indexing="ij")
x1ticks,x2ticks = x1mesh.flatten(),x2mesh.flatten()
xticks = torch.hstack([x1ticks[:,None],x2ticks[:,None]])
ymesh = u_2d(xticks).reshape(x1mesh.shape)
ylmesh = u_laplace_2d(xticks).reshape(x1mesh.shape)
fmesh = f_2d(xticks).reshape(x1mesh.shape)
fig = pyplot.figure(figsize=(12,8))
ax = np.empty((2,3),dtype=object)
_gs = gridspec.GridSpec(2,3) 
for i in range(3): ax[0,i] = fig.add_subplot(_gs[0,i],projection="3d")
for i in range(3): ax[1,i] = fig.add_subplot(_gs[1,i])
ax[0,0].plot_surface(x1mesh,x2mesh,ymesh,cstride=1,rstride=1,antialiased=False,cmap=cm.plasma)
ax[1,0].contourf(x1mesh,x2mesh,ymesh,antialiased=False,cmap=cm.plasma,levels=100)
ax[0,1].plot_surface(x1mesh,x2mesh,ylmesh,cstride=1,rstride=1,antialiased=False,cmap=cm.plasma);
ax[1,1].contourf(x1mesh,x2mesh,ylmesh,antialiased=False,cmap=cm.plasma,levels=100)
ax[0,2].plot_surface(x1mesh,x2mesh,fmesh,cstride=1,rstride=1,antialiased=False,cmap=cm.plasma);
ax[1,2].contourf(x1mesh,x2mesh,fmesh,antialiased=False,cmap=cm.plasma,levels=100)
ax[0,0].set_title(r"$u(\boldsymbol{x})$")
ax[0,1].set_title(r"$\Delta u(\boldsymbol{x})$")
ax[0,2].set_title(r"$f(\boldsymbol{x}) = C_1 \Delta u + C_2 \tau(u)$");